In [1]:
import tkinter as tk
from PIL import Image, ImageTk
import random
import sys
class TextRedirector(object):
    def __init__(self, widget, tag="stdout"):
        self.widget = widget
        self.tag = tag

    def write(self, str):
        self.widget.configure(state="normal")
        self.widget.insert("end", str, (self.tag,))
        self.widget.configure(state="disabled")
        self.widget.see("end")

    def flush(self):
        pass
class Characters:
    def __init__(self, name, health, attack, defense, speed, image):
        self.name = name
        self.hp = health
        self.ap = attack
        self.dp = defense
        self.sp = speed
        self.skills = {}
        self.jobclass = None
        self.image = image
        self.buff_tracker_dict = {}
        self.status_dict = {}
        self.statuses = []
        self.skill_choice = None

    def attack(self, enemy):
        print(f"{self.name} attacked.")
        # Assassin passive
        if enemy.jobclass == "Assassin":
            if enemy.passive():
                print(f"{enemy.name} dodged.")
                return None

        # Yasuo passive
        if enemy.jobclass == "Yasuo":
            if enemy.passive_active:
                print(f"{enemy.name}'s passive is active and damage has been mitigated.")
                enemy.attacks_taken += 1
                if enemy.attacks_taken == 2:
                    enemy.passive_active = False
                    enemy.passive_on_cooldown = True
                    enemy.attacks_taken = 0
                return None

        damage = round(self.ap / enemy.dp, 1)
        enemy.hp = round(enemy.hp - damage, 1)
        print(f"{self.name} dealt {damage} damage to {enemy.name}.")
        return damage

    def defend(self):
        dp_change = round(self.dp * .15, 1)
        self.buff_tracker(1, dp_change, "dp")
        self.dp = round(self.dp + dp_change, 1)
        print(f"{self.name} defended and raised defence stat.")

    def apply_status(self, status, damage, turns):
        if status in self.status_dict:
            self.status_dict[status]["Turns"] = turns
        else:
            self.status_dict[status] = {"Damage": damage, "Turns": turns}
            self.statuses.append(status)

    def damage_over_time(self):
        status_remover = []
        # Dealing damage and taking off turns from status
        for item in self.status_dict:
            damage = self.status_dict[item]["Damage"]
            self.hp -= damage
            self.status_dict[item]["Turns"] -= 1
            print(f"{self.name} took {damage} {item} damage ")
            if self.status_dict[item]["Turns"] == 0:
                status_remover.append(item)
        # removing statuses
        for removed in status_remover:
            self.status_dict.pop(removed)
            self.statuses.remove(removed)
        if self.hp < 0:
            self.hp = 0

    def skill(self, enemy):
        if self.skill_choice == 1:
            self.use_skill1(enemy)
        if self.skill_choice == 2:
            self.use_skill2(enemy)

    def buff_tracker(self, turns, change, stat):
        if len(self.buff_tracker_dict) == 0:
            self.buff_tracker_dict[1] = {"Turns": turns, "Stat Change": change, "Stat": stat}
        else:
            counter = list(self.buff_tracker_dict.keys())[-1]  # Ensure keys do not overide
            self.buff_tracker_dict[counter + 1] = {"Turns": turns, "Stat Change": change, "Stat": stat}

    def stat_reverter(self):
        remove_tracker = []
        for tracker in self.buff_tracker_dict:
            self.buff_tracker_dict[tracker]["Turns"] -= 1
            if self.buff_tracker_dict[tracker]["Turns"] == 0:
                stat_change = self.buff_tracker_dict[tracker]["Stat Change"]
                stat_category = self.buff_tracker_dict[tracker]["Stat"]
                current_stat = getattr(self, stat_category)
                ogstat_value = round(current_stat - stat_change, 2)
                setattr(self, stat_category, ogstat_value)
                remove_tracker.append(tracker)
        for tracker in remove_tracker:
            self.buff_tracker_dict.pop(tracker)


class Yasuo(Characters):
    def __init__(self, name):
        super().__init__(name, 60, 30, 2.5, 7, "Yasuo.png")
        self.mp = 30
        self.skills = {
            "Steel Tempest": {"Damage": 30, "Mana": 20},
            "Last Breath": {"Stat": "dp", "Stat Change": -.6, "Mana": 15, "Turns": 3},
        }
        self.skill_list = list(self.skills.keys())
        self.jobclass = "Yasuo"
        self.passive_active = False
        self.attacks_taken = 0
        self.passive_on_cooldown = False
        self.passive_turns_passed = 0


    def passive_cooldown(self):
        if self.passive_on_cooldown:
            self.passive_turns_passed += 1
            if self.passive_turns_passed == 2:
                self.passive_on_cooldown = False
                self.passive_turns_passed = 0

    def use_skill1(self, enemy):
        skill = self.skill_list[0]
        if self.mp >= self.skills[skill]["Mana"]:
            damage = self.skills[skill]["Damage"]
            enemy.hp = round(enemy.hp - damage, 1)
            self.mp -= self.skills[skill]["Mana"]
            print(f"{self.name} dealt {self.skills[skill]['Damage']} damage to {enemy.name}.")
            # passive
            if not self.passive_active and not self.passive_on_cooldown:
                self.passive_active = True
                print(f"{self.name}'s passive is active and will not take damage for 2 attacks.")
            elif self.passive_active:
                print(f"{self.name}'s passive is already active.")
            elif self.passive_on_cooldown:
                print(f"{self.name}'s passive is on cooldown")
        else:
            print("Magic points too low. Use another move")

    def use_skill2(self, enemy):
        skill = self.skill_list[1]
        if self.mp >= self.skills[skill]["Mana"]:
            turns = self.skills[skill]["Turns"]
            stat = self.skills[skill]["Stat"]
            stat_change = round(getattr(enemy, stat) * self.skills[skill]["Stat Change"], 1)
            new_stat = getattr(enemy, stat) + stat_change
            enemy.buff_tracker(turns, stat_change, stat)
            print(
                f"No damage done, enemy {stat} stat has been lowered by {abs(self.skills[skill]['Stat Change'] * 100)}% ")
            setattr(enemy, stat, round(new_stat, 1))
            self.mp -= self.skills[skill]["Mana"]
        else:
            print("Magic points too low. Use another move")
            game.player_turn()


class Assassin(Characters):
    def __init__(self, name):
        super().__init__(name, 65, 45, 1, 10, "Assassin.png")
        self.mp = 30
        self.skills = {
            "Assassinate": {"Damage": 1, "Mana": 25},
            "Freeze": {"Stat": "dp", "Stat Change": -.3, "Mana": 10, "Turns": 3}
        }
        self.skill_list = list(self.skills.keys())
        self.jobclass = "Assassin"


    def attack(self, enemy):
        super().attack(enemy)
        # bleed passive
        bleedchance = random.random()
        if bleedchance >= .7:
            enemy.apply_status("Bleed", 5, 2)
            print(f"{self.name} inflicted bleed damage to {enemy.name}")

    def passive(self):
        dodgechance = random.random()
        if dodgechance >= .4:
            return True
        return False

    def use_skill1(self, enemy):
        skill = self.skill_list[0]
        if self.mp >= self.skills[skill]["Mana"]:
            chance = random.random()
            if chance >= .9:
                damage = round(self.skills[skill]['Damage'] * enemy.hp, 1)
                enemy.hp = round(enemy.hp - damage, 1)
                print(f"{self.name} dealt {damage} damage to {enemy.name}")
            else:
                print(f"{self.name} missed.")
            self.mp -= self.skills[skill]["Mana"]

        else:
            print("Magic points too low. Use another move")
            game.player_turn()

    def use_skill2(self, enemy):
        skill = self.skill_list[1]
        if self.mp >= self.skills[skill]["Mana"]:
            turns = self.skills[skill]["Turns"]
            stat = self.skills[skill]["Stat"]
            stat_change = round(getattr(enemy, stat) * self.skills[skill]["Stat Change"], 1)
            new_stat = getattr(enemy, stat) + stat_change
            game.enemy.buff_tracker(turns, stat_change, stat)
            print(f"No damage done, enemy {stat} stat has been lowered by {abs(self.skills[skill]['Stat Change'] * 100)}%. Enemy frozen. ")
            setattr(enemy, stat, round(new_stat, 1))
            self.mp -= self.skills[skill]["Mana"]
            #skip enemy turn
            game.turn_end()
            game.skip_turn = True


class Mage(Characters):
    def __init__(self, name):
        super().__init__(name, 50, 50, 3, 5, "Mage.webp")
        self.mp = 75
        self.skills = {
            "Fireball": {"Damage": 20, "Mana": 40},
            "Weaken": {"Stat": "dp", "Stat Change": -.15, "Mana": 15, "Turns": 4}
        }
        self.skill_list = list(self.skills.keys())
        self.jobclass = "Mage"
        self.attack_counter = 0

    def attack(self, enemy):
        damage = super().attack(enemy)
        if self.attack_counter == 3:
            self.passive(enemy, damage)
            return None
        self.attack_counter += 1

    def passive(self, enemy, attackdamage):
        additional_damage = round(attackdamage * 1, 1)
        enemy.hp = round(enemy.hp - additional_damage, 1)
        print(f"{self.name} passive activated and dealt {additional_damage} additional damage.")
        self.attack_counter = 0

    def use_skill1(self, enemy):
        skill = self.skill_list[0]
        if self.mp >= self.skills[skill]["Mana"]:
            damage = self.skills[skill]['Damage']
            enemy.hp = round(enemy.hp - damage, 1)
            self.mp -= self.skills[skill]["Mana"]
            enemy.apply_status("Burn", 5, 3)
            print(
                f"{self.name} dealt {self.skills[skill]['Damage']} damage to {enemy.name} and a burn status has been applied.")
        else:
            print("Magic points too low. Use another move")
            game.player_turn()

    def use_skill2(self, enemy):
        skill = self.skill_list[1]
        if self.mp >= self.skills[skill]["Mana"]:
            turns = self.skills[skill]["Turns"]
            stat = self.skills[skill]["Stat"]
            stat_change = round(getattr(enemy, stat) * self.skills[skill]["Stat Change"], 1)
            new_stat = getattr(enemy, stat) + stat_change
            enemy.buff_tracker(turns, stat_change, stat)
            print(f"No damage done, enemy {stat} stat has been lowered by {abs(stat_change * 100)}% ")
            setattr(enemy, stat, round(new_stat, 1))
            self.mp -= self.skills[skill]["Mana"]
        else:
            print("Magic points too low. Use another move")
            game.player_turn()


class Game:
    def __init__(self):
        self.MAX_MP = 0
        self.player_classes = ["Yasuo", "Assassin", "Mage"]
        self.end_game = False
        self.restart = False
        self.player_selection = None
        self.skip_turn = True
        # GUI ELEMENTS
        self.winner = None
        self.confirmation = None



    def character_select(self):
        if self.player_selection == 1:
            self.player = Yasuo(self.player_name)
        if self.player_selection == 2:
            self.player = Assassin(self.player_name)
        if self.player_selection == 3:
            self.player = Mage(self.player_name)
        self.MAX_MP = self.player.mp


    def enemy_choice(self):
        enemy_choice = random.randint(1, 3)
        if enemy_choice == 1:
            self.enemy = Characters("Tank", 100, 10, 5, 2, "Tank.webp")
        elif enemy_choice == 2:
            self.enemy = Characters("Archer", 60, 15, 3, 7, "Archer.png")
        elif enemy_choice == 3:
            self.enemy = Characters("Barbarian", 75, 25, 4, 5, "Barbarian.png")


    def turn_order(self):
        if self.player.sp > self.enemy.sp:
            return 0
        elif self.enemy.sp > self.player.sp:
            return 1
        elif self.enemy.sp == self.player.sp:
            return 0

    def game_quit(self):
        print("Are you sure you want to quit.")
        while True:
            confirmation = input("Y/N: ")
            if confirmation.upper() in ["Y", "N"]:
                break
            print("Invalid input")
        if confirmation.upper() == "Y":
            self.end_game = True
        else:
            self.player_turn()

    def game_restart(self):
        if self.confirmation.upper() == "Y":
            self.restart = True
        else:
            self.player_turn()

    def player_turn(self):
        self.player.stat_reverter()
        # Call function
        if self.player_choice == 1:
            self.player.attack(self.enemy)
        elif self.player_choice == 2:
            self.player.defend()
        elif self.player_choice == 3:
            self.player.skill(self.enemy)
        elif self.player_choice == 4:
            self.game_restart()
        elif self.player_choice == 5:
            self.game_quit()

        self.update_label()

        if self.enemy.hp <= 0:
            self.enemy.hp = 0
            self.end_game = True
        if self.enemy.hp == 0:
            self.winner = self.player.name

    def enemy_turn(self):
        self.enemy.stat_reverter()
        print("Enemy turn.")
        enemy_turn_choice = random.randint(1, 2)
        if enemy_turn_choice == 1:
            self.enemy.attack(self.player)
        elif enemy_turn_choice == 2:
            self.enemy.defend()

        self.update_label()

        if self.player.hp <= 0:
            self.player.hp = 0
            self.end_game = True
        if self.player.hp == 0:
            self.winner = self.enemy.name

    def turn_end(self):
        # Regenerating player's mana points at end of each round
        self.player.mp += self.MAX_MP * .15
        if self.player.mp > self.MAX_MP:
            self.player.mp = self.MAX_MP
        # Checking turns since passive was deactived for Yasuo
        if self.player.jobclass == "Yasuo":
            self.player.passive_cooldown()
        # Status effects
        self.player.damage_over_time()
        self.enemy.damage_over_time()
        # Checking if enemy has died from status effect damage
        if self.player.hp <= 0:
            self.end_game = True
        if self.enemy.hp <= 0:
            self.end_game = True
        print("-----------------------------------------------")

    def update_label(self):
        self.player_stats.set(f"HP: {self.player.hp} \nDP: {self.player.dp} \nMP: {self.player.mp}/{self.MAX_MP} \nAP: {self.player.ap} \n SP: {self.player.sp}")
        self.enemy_stats.set(f"HP: {self.enemy.hp} \nDP: {self.enemy.dp} \nAP: {self.enemy.ap} \nSP: {self.enemy.sp}")
        if not self.player.statuses:
            self.player_statuses.set("No status effects")
        else:
            self.player_statuses.set(f"Player statuses: {self.player.statuses}")
        if not self.enemy.statuses:
            self.enemy_statuses.set("No status effects")
        else:
            self.enemy_statuses.set(f"Enemy statuses: {self.enemy.statuses}")

    def start(self):
        self.end_game = False
        self.update_label()
        turnorder = self.turn_order()
        if turnorder == 0:
            self.skip_turn = False
            self.player_turn()
            if self.enemy.hp == 0:
                return
            elif self.end_game:
                print("You quit the game.")
                return None
            elif self.restart:
                print("You restarted.")
            if self.skip_turn:
                return None
            self.enemy_turn()
            if self.player.hp == 0:
                return


            # ENEMY FIRST
        else:
            if not self.skip_turn:
                self.enemy_turn()
            else:
                self.skip_turn =False
            if self.player.hp == 0:
                return
            self.player_turn()
            if self.enemy.hp == 0:
                return
            elif self.end_game:
                print("You quit the game.")
                return None
            elif self.restart:
                print("You restarted.")

        self.turn_end()
        if self.enemy.hp == 0:
            print("Game over")
            print("You won!")
            self.winner = self.player.name
        elif self.player.hp == 0:
            print("Game over")
            print("You lost!")
            self.winner = self.enemy.name

    def gui(self):
        self.window = Window()






class Window(tk.Tk):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        # Adding a title to the window

        self.geometry("450x450")
        self.minsize(300,300)
        self.wm_title("Test Application")



        container = tk.Frame(self, height=400, width=600)
        container.pack(side="top", fill="both", expand=True)
        container.grid_rowconfigure(0, weight=1)
        container.grid_columnconfigure(0, weight=1)

        self.frames = {}
        #initalising frames and storing them in dict
        for classes in (MainScreen, CharacterSelectScreen, CombatScreen, EndScreen, SettingsScreen):
            frame = classes(container, self)
            self.frames[classes] = frame
            frame.grid(row=0,column=0,sticky="nsew")
            

        self.show_frame(MainScreen)
        self.mainloop()

    def show_frame(self, cont):
        frame = self.frames[cont]
        frame.tkraise()

    def resizer(self, event, titles, texts, factortitle, factortext, side):
        if side == "height":
            side = event.height
        else:
            side = event.width
        self.resize_title(titles, factortitle, side)
        self.resize_text(texts, factortext, side)


    def resize_title(self, title, factor, side):
        size = int(side/factor)
        title.config(font=("TkDfaultFont", size))

    def resize_text(self, texts, factor, side):
        for text in texts:
            size = int(side/factor)
            text.config(font=("TkDefaultFont", size))




class MainScreen(tk.Frame):
    def __init__(self, parent, mainwindow):
        super().__init__(parent)
        self.mainwindow = mainwindow
        #Variables
        self.photo = Image.open("logoimage.png")
        self.image = self.photo.resize((300, 300))
        self.image = ImageTk.PhotoImage(self.image)
        #Widgets
        self.title_label = tk.Label(
            self,
            name="title",
            text="DeadZone",
            font= ("TkDefaultFont", 20 , "bold" )
        )
        self.title_label.pack(padx=10, pady=5, fill="both", expand=True)
        self.canvas = tk.Canvas(
            self,
            width=300,
            height=300,

        )
        self.canvas.create_image(0, 0, image=self.image, anchor="nw", tags = "logo")
        self.canvas.pack( anchor=tk.CENTER)

        self.start_button = tk.Button(
            self,
            text="Start",
            command=lambda: self.start_button_func(),
            width=10
        )
        self.start_button.pack(padx= 100, fill=tk.X, expand=True)



        self.settings_button = tk.Button(
            self,
            text="Settings",
            width=10,
            command=lambda:mainwindow.show_frame(SettingsScreen)
        )
        self.bind("<Configure>", lambda event:  self.resize_elements(event))

        self.settings_button.pack(padx= 100, fill=tk.X, expand=True)

        self.quit_button = tk.Button(
            self,
            text="Quit",
            width=10,
            command=lambda:mainwindow.destroy()
        )
        self.quit_button.pack(padx=100,fill=tk.X,expand=True)

    def start_button_func(self):
        self.mainwindow.show_frame(CharacterSelectScreen)
        self.mainwindow.frames[CharacterSelectScreen].username.set("")
        self.mainwindow.geometry("800x600")
        self.mainwindow.minsize(500,400)


    def resize_logo(self, event):
        ratio = 300/500
        new_size = int(event.height * ratio)
        self.canvas.config(width=new_size, height=new_size)

        resized = self.photo.resize((new_size, new_size))
        self.image = ImageTk.PhotoImage(resized)
        self.canvas.delete("logo")
        self.canvas.create_image(0,0,image =self.image, anchor="nw", tags="logo" )

    def resize_elements(self, event):
        labels = [self.quit_button,self.settings_button,self.start_button]
        self.mainwindow.resizer(event, self.title_label, labels, 10,50, "height")
        self.resize_logo(event)
        self.start_button.pack_configure(padx=int(self.winfo_width()*.3))
        self.settings_button.pack_configure(padx=int(self.winfo_width() * .3))
        self.quit_button.pack_configure(padx=int(self.winfo_width() * .3))

class CharacterSelectScreen(tk.Frame):
    def __init__(self, parent, mainwindow):
        super().__init__(parent)
        #Variables
        self.username = tk.StringVar()
        self.mainwindow = mainwindow
        self.character_photo = None

        #Widgets
        self.selection_frame = tk.Frame(self)
        self.selection_frame.configure(highlightbackground="black", highlightthickness=1)
        self.selection_frame.place(relwidth=.4, relheight=1, relx=.01, rely=.5, anchor="w")

        self.title_label = tk.Label(
            self.selection_frame,
            text="Choose Your\nCharacter"
        )
        self.title_label.grid(row=0, column = 0, columnspan = 3)
        self.username_label = tk.Label(
            self.selection_frame,
            text="Enter your username: "
        )
        self.username_label.grid(row=1, column=0, columnspan=3)

        self.username_enter = tk.Entry(
            self.selection_frame,
            textvariable=self.username,
            width=300
        )

        self.username_enter.grid(row=2, column=0, columnspan=3, sticky="nw")

        self.yasuo_button = tk.Button(
            self.selection_frame,
            text="Yasuo",
            command=lambda: self.choose_button_function(1),
            width=5,
            height=1
        )
        self.yasuo_button.grid(row=3, column=0, sticky="nsew")

        self.assassin_button = tk.Button(
            self.selection_frame,
            text="Assassin",
            command=lambda: self.choose_button_function(2),
            width=5,
            height=1,

        )
        self.assassin_button.grid(row=3, column=1, sticky="nsew")

        self.mage_button = tk.Button(
            self.selection_frame,
            text="Mage",
            command=lambda: self.choose_button_function(3),
            width=5,
            height=1

        )
        self.mage_button.grid(row=3, column=2, sticky="nsew")

        self.character_frame = tk.Frame(self)
        self.character_frame.configure(highlightbackground="black", highlightthickness=1)
        self.character_frame.place(relwidth = .6, relheight = 1, relx=.99, rely=.5, anchor="e")

        self.character_jobclass = tk.Label(
            self.character_frame,
            text="Job Class: "
        )
        self.character_jobclass.grid(row=0, column=0,columnspan=3, sticky="nsew")

        self.character_canvas = tk.Canvas(
            self.character_frame,
            width = 100,
            height = 200
        )
        self.character_canvas.grid(row=1, column=0, columnspan=3,  sticky="nsew")

        self.character_stats = tk.Label(
            self.character_frame,
            text="STATS",
        )
        self.character_stats.grid(row=2, column=0, sticky="nsew")

        self.character_abilities = tk.Label(
            self.character_frame,
            text="ABILITIES"
        )
        self.character_abilities.grid(row=2, column=2, sticky="nsew")
        self.back_button = tk.Button(
            self.selection_frame,

            text="Go Back",
            command=lambda: self.back_button_func()
        )
        self.back_button.grid(row=5, column = 1, sticky="new")

        self.yasuo_button.bind("<Enter>", lambda event: self.hover_char(event,Yasuo(None)))
        self.yasuo_button.bind("<Leave>", lambda event: self.hover_remove(event))
        self.assassin_button.bind("<Enter>", lambda event: self.hover_char(event,Assassin(None)))
        self.assassin_button.bind("<Leave>", lambda event: self.hover_remove(event))
        self.mage_button.bind("<Enter>", lambda event: self.hover_char(event,Mage(None)))
        self.mage_button.bind("<Leave>", lambda event: self.hover_remove(event))
        self.bind("<Configure>", lambda event: self.resize_elements(event) )

    # Configuring grid for resizing
        for i in range(3):
            self.columnconfigure(i, weight = 1)
            self.rowconfigure(i, weight = 1)
            self.selection_frame.columnconfigure(i, weight=1)
            self.selection_frame.rowconfigure(i,weight=1)
            self.character_frame.columnconfigure(i, weight=1)
            self.character_frame.rowconfigure(i, weight=1)

        self.selection_frame.rowconfigure(4, weight =1)
        self.selection_frame.rowconfigure(5, weight =1)

        self.character_frame.rowconfigure(1, weight =2)



        for j in range(6, 8):
            self.selection_frame.rowconfigure(j,weight=3)


    def back_button_func(self):
        self.mainwindow.show_frame(MainScreen)
        self.mainwindow.geometry("450x450")
        self.mainwindow.minsize(300,300)

    def resize_elements(self, event):
        textlabels = [self.character_abilities,self.back_button, self.username_label, self.yasuo_button, self.assassin_button, self.mage_button, self.username_label, self.character_jobclass, self.character_stats]
        self.mainwindow.resizer(event, self.title_label, textlabels, 30, 70, "width")
        self.image_resize(event)

    def image_resize(self, event):
        if self.character_photo == None:
            return
        width = self.character_canvas.winfo_width()
        height = self.character_canvas.winfo_height()
        self.character_image = self.character_photo.resize((width, height))
        self.character_image = ImageTk.PhotoImage(self.character_image)
        self.character_canvas.create_image(0, 0, image=self.character_image, anchor="nw", tags="character")
        self.character_canvas.grid_configure(padx=self.winfo_width()/3000)
    def hover_char(self, event, character):
        #Jobclass
        self.character_jobclass.config(text=f"Job Class: {character.jobclass}")
        #Image
        self.character_photo = Image.open(character.image)
        width = self.character_canvas.winfo_width()
        height = self.character_canvas.winfo_height()
        self.character_image =  self.character_photo.resize((width, height))
        self.character_image = ImageTk.PhotoImage(self.character_image)
        self.character_canvas.create_image(0, 0, image=self.character_image, anchor="nw", tags = "character")
        self.character_canvas.grid_configure(padx=self.winfo_width()*.2)
        #Stats
        character_stats = f" STATS\n HP: {character.hp} \nAP: {character.ap} \nDP: {character.dp} \nSP: {character.sp} \nMP: {character.mp}"
        self.character_stats.config(text=character_stats)
        #Abilities
        skill1 = character.skill_list[0]
        skill2 = character.skill_list[1]
        mana1 = character.skills[skill1]["Mana"]
        mana2 = character.skills[skill2]["Mana"]
        abilities = f"ABILITIES\n{skill1} MP Cost: {mana1}\n{skill2} MP Cost: {mana2}"
        self.character_abilities.config(text=abilities)



    def hover_remove(self, event):
        self.character_photo=None
        self.character_jobclass.config(text="Job Class: ")
        self.character_canvas.delete("character")
        self.character_stats.config(text="STATS")
        self.character_abilities.config(text="ABILITIES")

    def choose_button_function(self, choice):
        self.mainwindow.geometry("800x500")
        game.player_selection = choice
        game.player_name = self.username.get()
        game.character_select()
        game.enemy_choice()
        #Username
        self.mainwindow.frames[CombatScreen].username_label.config(text=game.player_name)
        self.mainwindow.frames[CombatScreen].enemy_username.config(text=game.enemy.name)
        #Skills
        skill1 = game.player.skill_list[0]
        skill2 = game.player.skill_list[1]
        self.mainwindow.frames[CombatScreen].skill1_name.set(f"{skill1} Cost: {game.player.skills[skill1]['Mana']} MP")
        self.mainwindow.frames[CombatScreen].skill2_name.set(f"{skill2} Cost: {game.player.skills[skill2]['Mana']} MP")

        #Images
        frame = self.mainwindow.frames[CombatScreen]
        frame.player_photo = Image.open(game.player.image)
        frame.player_image = frame.player_photo.resize((400,800))
        frame.player_image = ImageTk.PhotoImage(frame.player_image)
        frame.mainwindow.frames[CombatScreen].player_canvas.create_image(0,0, image=frame.player_image, anchor="nw", tag="player_image")
        frame.enemy_photo = Image.open(game.enemy.image)
        frame.enemy_image = frame.enemy_photo.resize((400,800))
        frame.enemy_image = ImageTk.PhotoImage(frame.enemy_image)
        frame.enemy_canvas.create_image(0,0, image=frame.enemy_image, anchor="nw", tag="enemy_image")
        game.update_label()
        frame.text_box.config(state="normal")
        frame.text_box.delete(1.0, "end")
        frame.text_box.config(state="disabled")

        self.mainwindow.geometry("1067x600")
        self.mainwindow.minsize(889, 500)
        self.mainwindow.show_frame(CombatScreen)
class CombatScreen(tk.Frame):
    def __init__(self, parent, mainwindow):
        super().__init__(parent)
        #Variables
        self.mainwindow = mainwindow

        self.player_photo = Image.open("Yasuo.png")
        self.enemy_photo = Image.open("Yasuo.png")
        self.skill1_name = tk.StringVar()
        self.skill2_name = tk.StringVar()
        game.player_stats = tk.StringVar()
        game.enemy_stats = tk.StringVar()
        game.winner = tk.StringVar()
        game.player_statuses = tk.StringVar()
        game.enemy_statuses = tk.StringVar()
        #Widgets
        self.title_label = tk.Label(
            self,
            text="Combat Screen"
        )
        self.title_label.grid(row=0, column = 0,columnspan=3,  sticky="nsew")

        #PLAYER ELEMENTS
        self.username_label = tk.Label(
            self,
            text= ""
        )
        self.username_label.grid(row=1, column=0, sticky="nsew")
        self.player_canvas = tk.Canvas(
            self,
            width= 180,
            height = 250
        )
        self.player_canvas.grid(row=2,rowspan=2, column=0, sticky="ns")
        self.player_statuses_label = tk.Label(
            self,
            textvariable=game.player_statuses
        )
        self.player_statuses_label.grid(row=4, column=0, sticky="nsew")
        self.player_stats_label = tk.Label(
            self,
            textvariable= game.player_stats
        )
        self.player_stats_label.grid(row=5, column=0, sticky="nsew")

        self.attack_button = tk.Button(
            self,
            text="Attack",
            command= lambda: self.player_button(1)
        )
        self.attack_button.grid(row=6, column=0, sticky="nsew")

        self.defend_button = tk.Button(
            self,
            text="Defend",
            command=lambda: self.player_button(2)
        )
        self.defend_button.grid(row=7, column = 0, sticky="nsew")

        self.skill1_button = tk.Button(
            self,
            textvariable = self.skill1_name,
            command=lambda: self.player_button(3, 1),
            width=30

        )
        self.skill1_button.grid(row=8, column=0, sticky="nsew")
        self.skill2_button = tk.Button(
            self,
            textvariable = self.skill2_name,
            command=lambda: self.player_button(3,2)
        )
        self.skill2_button.grid(row=9, column=0, sticky="nsew")

        self.restart_button = tk.Button(
            self,
            text="Restart",
            command=lambda: self.player_button(4)
        )
        self.restart_button.grid(row=10,column=0, sticky="nsew")
        
        self.quit_button = tk.Button(
            self,
            text="Quit",
            command = lambda: mainwindow.destroy()
        )
        self.quit_button.grid(row=11, column=0, sticky="nsew")

        #ENEMY ELEMENTS
        self.enemy_username = tk.Label(
            self,
            text=""
        )
        self.enemy_username.grid(row=1, column=2, sticky="sew")

        self.enemy_canvas = tk.Canvas(
            self,
            width=180,
            height=250,
        )
        self.enemy_canvas.grid(row=2,rowspan=2, column=2, sticky="ns")

        self.enemy_statuses_label = tk.Label(
            self,
            textvariable=game.enemy_statuses
        )
        self.enemy_statuses_label.grid(row=4, column=2, sticky="nsew")

        self.enemy_stats_label = tk.Label(
            self,
            textvariable=game.enemy_stats
        )
        self.enemy_stats_label.grid(row=5, column = 2, sticky="nsew")

        self.text_box = tk.Text(
            self,
            width=60,
            height=20,
            state=tk.DISABLED
        )
        self.text_box.grid(row=0,rowspan=11, column= 4, columnspan=8, sticky="nsew")
        sys.stdout = TextRedirector(self.text_box, "stdout")

        #Configuring grid for resizing
        for i in range(11):
            self.grid_columnconfigure(i, weight = 1)
            self.grid_rowconfigure(i, weight=1)


        self.bind("<Configure>", lambda event: self.resize_elements(event))
        
    def quit_button_func(self):
        self.mainwindow.destroy()

    def restart_popup(self):
        def yes_func():
            game.confirmation = "Y"
            popup.destroy()

        def no_func():
            game.confirmation = "N"
            popup.destroy()

        popup = tk.Toplevel(self.mainwindow)
        popup.geometry("400x200")
        popup.minsize(200, 100)
        popup.maxsize(400,200)
        popup.title("Restart Confirmation")
        popup_label = tk.Label(
            popup,
            text = "Are you sure you want to restart?",
            font=("TkDefaultFont", 10, "bold")
        )
        popup_label.grid(row=0, column = 0, columnspan=2, sticky="ew")
        yes_button = tk.Button(
            popup,
            text = "Yes",
            command= lambda: [yes_func(), self.window_resize()]
        )

        yes_button.grid(row=1, column=0, sticky='ew', padx=20)
        no_button = tk.Button(
            popup,
            text="No",
            command= lambda: no_func()
        )
        no_button.grid(row=1, column=1, sticky="ew", padx =20)

        for i in range(2):
            popup.rowconfigure(i, weight=1)
            popup.columnconfigure(i,weight=1)

        popup.mainloop()

    def window_resize(self):
        self.mainwindow.show_frame(CharacterSelectScreen)
        self.mainwindow.geometry("800x600")
        self.mainwindow.minsize(500,400)



    def player_button(self, choice, skillchoice = None):
        if choice == 3:
            game.player_choice = choice
            game.player.skill_choice = skillchoice
            skill = game.player.skill_list[skillchoice-1]
            if game.player.mp < game.player.skills[skill]["Mana"]:
                print("Not enough MP. Choose a different move.")
                return None
        elif choice == 4:
            self.restart_popup()
            game.player_choice = choice
            return None
        else:
            game.player_choice = choice
        game.start()

        if game.end_game:
            if game.winner != game.player.name:
                self.mainwindow.frames[EndScreen].title.config(text = "Game Over!\nYou Lost!")
            else:
                self.mainwindow.frames[EndScreen].title.config(text="Game Over!\nYou Won!")

            self.mainwindow.frames[EndScreen].winner_label.config(text=f"{game.winner} Won!")
            self.mainwindow.show_frame(EndScreen)
    def resize_elements(self, event):
        labels = [self.username_label, self.player_statuses_label, self.player_stats_label, self.attack_button,
                  self.defend_button, self.skill1_button, self.skill2_button, self.restart_button,
                  self.enemy_username, self.enemy_stats_label, self.enemy_statuses_label, self.enemy_stats_label]
        self.mainwindow.resizer(event, self.title_label, labels, 30, 100, "width")
        self.resize_image(event)


    def resize_image(self, event):
        new_width = int(self.player_canvas.winfo_width())
        new_height = int(self.player_canvas.winfo_height())
        playerphoto = self.player_photo
        resized = playerphoto.resize((new_width, new_height))
        self.player_image = ImageTk.PhotoImage(resized)
        self.player_canvas.delete("player_image")
        self.player_canvas.create_image(0, 0, image=self.player_image, anchor="nw", tags="player_image")

        enemyphoto = self.enemy_photo
        resized = enemyphoto.resize((new_width,new_height))
        self.enemy_image = ImageTk.PhotoImage(resized)
        self.enemy_canvas.delete("enemy_image")
        self.enemy_canvas.create_image(0, 0, image=self.enemy_image, anchor="nw", tags="enemy_image")

class EndScreen(tk.Frame):
    def __init__(self, parent, mainwindow):
        super().__init__(parent)
        self.mainwindow = mainwindow
        self.title = tk.Label(
            self,
            text="Game Over",
            font=("TkDefaultFont", 40)
        )
        self.title.pack(side="top", expand=True)
        self.winner_label = tk.Label(
            self,
            text=""
        )
        self.winner_label.pack(expand=True)
        self.back_button = tk.Button(
            self,
            text="Go Back",
            command=lambda: self.back_button_func()
        )
        self.back_button.pack(side="bottom", pady = 30, expand=True)

        self.bind("<Configure>", lambda event:self.resize_elements(event))

    def resize_elements(self, event):
        labels = [self.winner_label, self.back_button]
        self.mainwindow.resizer(event, self.title, labels, 30, 60, "width")
    def back_button_func(self):
        self.mainwindow.show_frame(MainScreen)
        self.mainwindow.geometry("450x450")
        self.mainwindow.minsize(300,300)

class SettingsScreen(tk.Frame):
    def __init__(self, parent, mainwindow):
        super().__init__(parent)
        self.mainwindow = mainwindow

        self.title_label = tk.Label(
            self,
            text="Settings"
        )
        self.title_label.grid(row=0, column=0,columnspan=5, sticky="nsew")
        # Audio
        self.audio_val = tk.IntVar()
        self.audio_val.set(1)
        self.audio_label = tk.Label(
            self,
            text="Audio",
        )
        self.audio_label.grid(row=1, column=0, sticky="nsew")
        
        self.audio_on = tk.Radiobutton(
            self,
            text="On",
            variable=self.audio_val,
            value=1
        )
        self.audio_on.grid(row=1, column=2, sticky="nsew")

        self.audio_off = tk.Radiobutton(
            self,
            text="Off",
            variable=self.audio_val,
            value=0
        )
        self.audio_off.grid(row=1, column=3, sticky="nsew")

        # Language
        self.languages = ["English", "Chinese", "Malay", "Hindi"]
        self.current_language = tk.StringVar()
        self.current_language.set(self.languages[0])
        self.language_dropdown = tk.OptionMenu(
            self,
            self.current_language,
            *self.languages)
        self.language_dropdown.grid(row=2, column=2, sticky="ew")

        self.language_text = tk.Label(
            self,
            text="Language"
        )
        self.language_text.grid(row=2, column=0, sticky="ew")

        # Difficulty
        self.difficulty_text = tk.Label(
            self,
            text="Difficulty"
        )
        self.difficulty_text.grid(row=3, column=0, sticky="ew")

        self.difficulty_scale = tk.Scale(
            self,
            from_=1,
            to=3,
            orient=tk.HORIZONTAL
        )
        self.difficulty_scale.grid(row=3, column=2, sticky="ew")

        self.settings_back = tk.Button(
            self,
            text="Back",
            command=lambda: mainwindow.show_frame(MainScreen)
        )
        self.settings_back.grid(row=5, column = 2)

        for i in range(5):
            self.grid_columnconfigure(i, weight=1)
            self.grid_rowconfigure(i, weight=1)

        self.bind("<Configure>", lambda event: self.resize_elements(event))

    def resize_elements(self, event):
        labels = [self.audio_label, self.audio_on, self.audio_off, self.language_text, self.language_dropdown, self.difficulty_text, self.settings_back]
        self.mainwindow.resizer(event, self.title_label, labels, 20,40, "width")


        



game = Game()
game.gui()

